# ReACT Agent Flow - Local LLM SDK

This notebook demonstrates a **ReACT (Reasoning, Action, Observation)** agent implementation using the `local_llm_sdk` with LM Studio.

## What is ReACT?

ReACT is a paradigm where an AI agent:
1. **Thinks/Reasons** about the task
2. **Acts** by using tools to gather information or perform actions
3. **Observes** the results of its actions
4. **Iterates** until the task is complete

## Our Tools

- **Python Executor**: Execute arbitrary Python code safely
- **Filesystem Operations**: Create directories, read/write files, list contents
- **Math Calculator**: Perform mathematical operations

## Example Task
**"Implement a sorting algorithm with Python primitives, test it and benchmark against standard sorting algorithms"**

---

## 1. Setup - Import and Initialize Client

In [1]:
# Install the package
!pip install -e .. --force-reinstall

Obtaining file:///home/maheidem/gen-ai-api-study
  Preparing metadata (setup.py) ... done
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached pydantic-2.11.9-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x8

In [2]:
# Import the SDK and create client with tools
from local_llm_sdk import LocalLLMClient, create_chat_message
import json
import time
from datetime import datetime

# Create client with LM Studio
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"
)

# Register our built-in tools (including the new ReACT tools)
client.register_tools_from(None)  # Loads from builtin

print(f"✅ Client initialized with {len(client.tools.list_tools())} tools:")
for tool in client.tools.list_tools():
    print(f"  • {tool}")

✅ Client initialized with 6 tools:
  • char_counter
  • math_calculator
  • get_weather
  • text_transformer
  • execute_python
  • filesystem_operation


## 2. ReACT Agent Implementation

In [3]:
class ReACTAgent:
    """
    ReACT Agent that thinks, acts, and observes iteratively.
    Uses LM Studio with function calling to solve complex tasks.
    """

    def __init__(self, client, max_iterations=10, verbose=True):
        self.client = client
        self.max_iterations = max_iterations
        self.verbose = verbose
        self.conversation_history = []
        self.iteration_count = 0
        
    def think_and_act(self, task_prompt, system_prompt=None):
        """
        Execute the ReACT loop for a given task.
        """
        if system_prompt is None:
            system_prompt = self._get_default_system_prompt()
            
        # Initialize conversation with system and user message
        self.conversation_history = [
            create_chat_message("system", system_prompt),
            create_chat_message("user", task_prompt)
        ]
        
        self.iteration_count = 0
        
        print(f"🎯 **TASK**: {task_prompt}")
        print("=" * 80)
        
        while self.iteration_count < self.max_iterations:
            self.iteration_count += 1
            
            if self.verbose:
                print(f"\n🔄 **Iteration {self.iteration_count}**")
                print("-" * 40)
            
            # Get response from the model
            try:
                response = self.client.chat(
                    self.conversation_history,
                    temperature=0.1,  # Lower temperature for more focused reasoning
                    max_tokens=2000
                )
                
                # Check if this is a ChatCompletion object or string
                if hasattr(response, 'choices'):
                    message = response.choices[0].message
                    content = message.content
                    tool_calls = getattr(message, 'tool_calls', None)
                else:
                    content = str(response)
                    tool_calls = None
                
                # Add assistant's response to conversation
                assistant_message = create_chat_message("assistant", content)
                if tool_calls:
                    assistant_message.tool_calls = tool_calls
                self.conversation_history.append(assistant_message)
                
                if self.verbose:
                    print(f"🤖 **Thinking**: {content}")
                
                # Execute tool calls if any
                if tool_calls:
                    for tool_call in tool_calls:
                        tool_name = tool_call.function.name
                        tool_args = json.loads(tool_call.function.arguments)
                        
                        if self.verbose:
                            print(f"\n🔧 **Action**: {tool_name}({tool_args})")
                        
                        # Execute the tool
                        tool_result = self.client.tools.execute(tool_name, tool_args)
                        
                        if self.verbose:
                            print(f"📊 **Observation**: {tool_result[:200]}{'...' if len(tool_result) > 200 else ''}")
                        
                        # Add tool result to conversation
                        tool_message = create_chat_message("tool", tool_result)
                        tool_message.tool_call_id = tool_call.id
                        self.conversation_history.append(tool_message)
                else:
                    # No tool calls, check if task is complete
                    if any(phrase in content.lower() for phrase in ["task complete", "finished", "done", "completed the task"]):
                        print(f"\n✅ **Task completed in {self.iteration_count} iterations!**")
                        break
                    
                    # If no tools and not complete, ask for next action
                    follow_up = "Please continue with the next step or let me know if the task is complete."
                    self.conversation_history.append(create_chat_message("user", follow_up))
                
            except Exception as e:
                print(f"❌ **Error in iteration {self.iteration_count}**: {e}")
                break
        
        if self.iteration_count >= self.max_iterations:
            print(f"\n⚠️ **Maximum iterations ({self.max_iterations}) reached!**")
        
        return self.conversation_history
    
    def _get_default_system_prompt(self):
        return """You are a ReACT agent that thinks step by step and uses tools to solve tasks.

FOLLOW THIS PATTERN:
1. **Think** about what you need to do
2. **Act** by calling appropriate tools
3. **Observe** the results
4. **Iterate** until the task is complete

Available tools:
- execute_python: Run Python code and get results
- filesystem_operation: Create directories, read/write files, list contents
- math_calculator: Perform mathematical operations
- char_counter: Count characters in text
- get_weather: Get weather information (mock)
- text_transformer: Transform text case

When the task is complete, clearly state "Task completed" in your response.

Be methodical and explain your reasoning for each step."""

print("✅ ReACT Agent class defined!")

✅ ReACT Agent class defined!


## 3. Create Agent Instance

In [4]:
# Create our ReACT agent
agent = ReACTAgent(client, max_iterations=15, verbose=True)

print("🤖 ReACT Agent ready for action!")

🤖 ReACT Agent ready for action!


## 4. Example Task: Sorting Algorithm Implementation

Let's give our agent a complex task that requires multiple steps:
1. Understanding the requirements
2. Creating project structure
3. Implementing the algorithm
4. Testing the implementation
5. Benchmarking against standard algorithms
6. Saving results

In [5]:
# Define our complex task
task = """
Implement a sorting algorithm with Python primitives, test it and benchmark against standard sorting algorithms.

Requirements:
1. Check if a 'sorting_algorithm' folder exists, if not create it
2. Implement bubble sort algorithm in Python using only basic primitives (no built-in sort)
3. Save the implementation as 'bubble_sort.py' in the sorting_algorithm folder
4. Create test cases to verify the algorithm works correctly
5. Benchmark it against Python's built-in sorted() function with different array sizes
6. Save benchmark results to a file

Make sure to handle edge cases and provide clear output.
"""

# Execute the task
conversation = agent.think_and_act(task)

🎯 **TASK**: 
Implement a sorting algorithm with Python primitives, test it and benchmark against standard sorting algorithms.

Requirements:
1. Check if a 'sorting_algorithm' folder exists, if not create it
2. Implement bubble sort algorithm in Python using only basic primitives (no built-in sort)
3. Save the implementation as 'bubble_sort.py' in the sorting_algorithm folder
4. Create test cases to verify the algorithm works correctly
5. Benchmark it against Python's built-in sorted() function with different array sizes
6. Save benchmark results to a file

Make sure to handle edge cases and provide clear output.


🔄 **Iteration 1**
----------------------------------------
🤖 **Thinking**: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "create_directory", "path": "sorting_algorithm"}[/THINK]The directory 'sorting_algorithm' has been created.

Now let's implement the bubble sort algorithm using only basic primitives (no built-in sort functions).

We'll define a function that takes a 

## 5. View Generated Files

In [6]:
# Let's see what files were created
import os
from pathlib import Path

# Check if the sorting_algorithm directory was created
sorting_dir = Path("sorting_algorithm")
if sorting_dir.exists():
    print("📁 Files in sorting_algorithm directory:")
    print("=" * 40)
    
    for file_path in sorting_dir.iterdir():
        print(f"📄 {file_path.name}")
        
        # Show content of small files
        if file_path.is_file() and file_path.stat().st_size < 2000:
            print(f"\nContent of {file_path.name}:")
            print("-" * 30)
            with open(file_path, 'r') as f:
                print(f.read())
            print("-" * 30)
        print()
else:
    print("❌ sorting_algorithm directory not found")

📁 Files in sorting_algorithm directory:
📄 bubble_sort.py

Content of bubble_sort.py:
------------------------------
def bubble_sort(arr):
    n = len(arr)
    # Make a copy to avoid modifying the original
    arr_copy = arr.copy()
    for i in range(n):
        # Flag to check if any swap happened in this pass
        swapped = False
        for j in range(0, n-i-1):
            if arr_copy[j] > arr_copy[j+1]:
                # Swap the elements
                arr_copy[j], arr_copy[j+1] = arr_copy[j+1], arr_copy[j]
                swapped = True
        # If no swaps happened, array is already sorted
        if not swapped:
            break
    return arr_copy

# Test cases to verify correctness
def test_bubble_sort():
    test_cases = [
        ([], []),
        ([1], [1]),
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),  # Already sorted
        ([5, 2, 7, 3, 4, 6, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9]),
        ([1, 1, 1], [1, 1, 1]),      # With duplicates
    ]

    fo

## 6. Test a Simpler Task

In [8]:
# Let's try a simpler task to test our ReACT agent
simple_task = """
Create a simple Python script that:
1. Creates a 'test_project' directory
2. Writes a Python script called 'hello.py' that prints "Hello from ReACT!"
3. Executes the script to verify it works
4. Lists the contents of the test_project directory
"""

print("🎯 Testing with a simpler task...")
print("=" * 50)

# Create a new agent instance for the simple task
simple_agent = ReACTAgent(client, max_iterations=8, verbose=True)
simple_conversation = simple_agent.think_and_act(simple_task)

🎯 Testing with a simpler task...
🎯 **TASK**: 
Create a simple Python script that:
1. Creates a 'test_project' directory
2. Writes a Python script called 'hello.py' that prints "Hello from ReACT!"
3. Executes the script to verify it works
4. Lists the contents of the test_project directory


🔄 **Iteration 1**
----------------------------------------
🤖 **Thinking**: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "write_file", "path": "test_project/hello.py", "content": "print(\"Hello from ReACT!\")"}

🔄 **Iteration 2**
----------------------------------------
🤖 **Thinking**: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "list_contents", "path": "test_project"}

🔄 **Iteration 3**
----------------------------------------
🤖 **Thinking**: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "create_directory", "path": "test_project"}

🔄 **Iteration 4**
----------------------------------------
🤖 **Thinking**: [TOOL_CALLS]filesystem_operation[ARGS]{"operation": "list_contents", "path

## 7. Advanced ReACT Example: Data Analysis Task

In [ ]:
# More complex task involving data analysis
data_task = """
Create a data analysis project:
1. Create a 'data_analysis' folder
2. Generate sample data (list of numbers) using Python
3. Calculate statistics: mean, median, standard deviation
4. Save the data and statistics to files
5. Create a summary report
"""

print("📊 Starting data analysis task...")
print("=" * 50)

data_agent = ReACTAgent(client, max_iterations=12, verbose=True)
data_conversation = data_agent.think_and_act(data_task)

## 8. Conversation Analysis

In [ ]:
# Analyze the conversation from our first task
def analyze_conversation(conversation):
    """Analyze the ReACT conversation to extract insights."""
    
    print("🔍 **Conversation Analysis**")
    print("=" * 50)
    
    message_types = {}
    tool_calls_made = []
    
    for i, msg in enumerate(conversation):
        role = msg.role
        message_types[role] = message_types.get(role, 0) + 1
        
        # Track tool calls
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                tool_calls_made.append(tool_call.function.name)
    
    print(f"📈 **Message Statistics:**")
    for role, count in message_types.items():
        print(f"  • {role.title()}: {count} messages")
    
    print(f"\n🔧 **Tools Used:**")
    if tool_calls_made:
        from collections import Counter
        tool_counts = Counter(tool_calls_made)
        for tool, count in tool_counts.items():
            print(f"  • {tool}: {count} times")
    else:
        print("  • No tools were used")
    
    print(f"\n📝 **Total Conversation Length:** {len(conversation)} messages")

# Analyze our main conversation
if 'conversation' in locals():
    analyze_conversation(conversation)
else:
    print("No conversation to analyze yet. Run the tasks above first!")

## 9. Save Conversation Log

In [ ]:
# Save our conversation for later analysis
def save_conversation_log(conversation, filename):
    """Save conversation history to a file."""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"conversation_log_{timestamp}_{filename}.txt"
    
    with open(log_file, 'w') as f:
        f.write(f"ReACT Agent Conversation Log\n")
        f.write(f"Generated: {datetime.now()}\n")
        f.write("=" * 60 + "\n\n")
        
        for i, msg in enumerate(conversation):
            f.write(f"Message {i+1} - {msg.role.upper()}\n")
            f.write("-" * 30 + "\n")
            f.write(f"{msg.content}\n")
            
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                f.write("\nTool Calls:\n")
                for tool_call in msg.tool_calls:
                    f.write(f"  • {tool_call.function.name}({tool_call.function.arguments})\n")
            
            f.write("\n" + "=" * 60 + "\n\n")
    
    return log_file

# Save conversation if it exists
if 'conversation' in locals() and conversation:
    log_file = save_conversation_log(conversation, "sorting_algorithm")
    print(f"💾 Conversation saved to: {log_file}")
else:
    print("No conversation to save yet.")

## 10. ReACT Agent Evaluation

In [ ]:
# Quick evaluation of the agent's performance
def evaluate_agent_performance():
    """Evaluate how well our ReACT agent performed."""
    
    print("🏆 **ReACT Agent Performance Evaluation**")
    print("=" * 50)
    
    # Check if files were created
    success_metrics = {
        "Sorting Algorithm Directory": Path("sorting_algorithm").exists(),
        "Test Project Directory": Path("test_project").exists(),
        "Data Analysis Directory": Path("data_analysis").exists(),
    }
    
    for metric, passed in success_metrics.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        print(f"  • {metric}: {status}")
    
    success_rate = sum(success_metrics.values()) / len(success_metrics) * 100
    print(f"\n📊 **Overall Success Rate: {success_rate:.1f}%**")
    
    if success_rate >= 80:
        print("🎉 Excellent performance!")
    elif success_rate >= 60:
        print("👍 Good performance!")
    else:
        print("🔧 Needs improvement.")

evaluate_agent_performance()

## Summary

This notebook demonstrated a complete **ReACT Agent** implementation using the `local_llm_sdk`:

### ✅ What We Accomplished:

1. **Enhanced Tool System** - Added Python execution and filesystem tools
2. **ReACT Agent Class** - Implemented reasoning → action → observation loop
3. **Complex Task Execution** - Sorting algorithm implementation and testing
4. **Iterative Problem Solving** - Agent breaks down tasks into manageable steps
5. **Tool Integration** - Seamless use of multiple tools in sequence
6. **Conversation Tracking** - Full conversation history and analysis

### 🎯 Key Features:

- **Safety**: Sandboxed Python execution and filesystem restrictions
- **Flexibility**: Configurable iteration limits and verbosity
- **Observability**: Detailed logging of thoughts, actions, and observations
- **Extensibility**: Easy to add new tools and capabilities

### 🚀 Next Steps:

- Add more specialized tools (web scraping, API calls, etc.)
- Implement memory persistence across sessions
- Add error recovery and retry mechanisms
- Create task-specific agent templates
- Build a web interface for the ReACT agent

### 📚 Learn More:

- [ReACT Paper](https://arxiv.org/abs/2210.03629) - Original research
- [LM Studio Documentation](https://lmstudio.ai/docs) - Local LLM setup
- [Local LLM SDK Documentation](../README.md) - Our SDK details

---

🎉 **Congratulations!** You now have a fully functional ReACT agent that can solve complex, multi-step programming tasks autonomously!